# How To Create a Vertex Search AI Data Store Using Python

This notebook outlines how to create an ustructured data store in Vertex AI Search. In this example we will create a Vertex AI Search data store and create some documents from a GCS bucket. After creating the data store, we will perform a query and optionally pass the query and the results to Gemini-Pro to produce the final answer.

## Prepare the python development environment

First, let's identify any project specific variables to customize this notebook to your GCP environment. Change YOUR_PROJECT_ID with your own GCP project ID.

In [ ]:
PROJECT_ID = 'YOUR_PROJECT_ID'
REGION = 'us-central1'
LOCATION = 'global'
DSENGINE = 'YOUR_DS_ENGINE_ID'

Install any needed python modules from our requirements.txt file. Most Vertex Workbench environments include all the packages we'll be using, but if you are using an external Jupyter Notebook or require any additional packages for your own needs, you can simply add them to the included requirements.txt file an run the folloiwng commands.

In [ ]:
#pip install -r requirements.txt

Now we will import all required modules

In [ ]:
import vertexai

from typing import List
from google.api_core.client_options import ClientOptions

from google.cloud import discoveryengine_v1beta as discoveryengine


## Instantiate Vertex AI ojbect

Define a function to search the data store based on the information from the user prompt

In [ ]:
def search_sample(project_id, location, engine_id, search_query) -> List[discoveryengine.SearchResponse]:

    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    
    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)
    
    # The full resource name of the search app serving config
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"
    
    
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
            ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            model_prompt_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelPromptSpec(
                preamble=""
            ),
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="gemini-1.5-flash-001/answer_gen/v1",
            ),
        ),
    )
    
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        #page_size=1,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    #print(response)

    return response

In [ ]:
search_prompt = 'Tell me more about EXT11'
#search_prompt = 'How many Old English items do I have?'

In [ ]:
search_result = search_sample(PROJECT_ID, LOCATION, DSENGINE, search_prompt)

Print the full output of the response

In [ ]:
#print(search_result)

Print the summary text

In [ ]:
print(search_result.summary.summary_text)

Print the link to the supporting documents

In [ ]:
#-- Print just the first returned document --#
#print(search_result.results[0].document.derived_struct_data['link'])

#-- Print the full list of returned documents --#
for i in search_result.results:
    print(i.document.derived_struct_data['link'])